Osnabrück University - Machine Learning (Summer Term 2020) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack, Axel Schaffland

# Exercise Sheet 05b: Dimension Reduction

## Introduction

This week's sheet should be solved and handed in before the end of **Saturday, June 06, 2020**. If you need help (and Google and other resources were not enough), feel free to contact your groups designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Assignment 1: Covariance and autocorrelation matrix [5 Points]

**a)** Recap: What is the covariance matrix? What do the different entries of the matrix mean? 

The covariance matrix tells you how much one feature in your data set covariates with another feature in the data set.
On the diagonal, you just have the variance of the individual features and the non-diagonal elements tell you how feature i covariates with feature j.
A positive covariance value means that high values for the first feature correspond to high values for the second one. A negative covariance value on the other hand indicates that high values for the first feature correspond to low values in the second feature (0 means no correspondence at all).
If the covariance matrix is normalized, it results in the correlation matrix with values between -1 and 1 (correlation between features).

**b)** What can the covariance matrix be used for? Name at least two applications.

- PCA: The PCs are eigenvectors of the covariance matrix
- Stochastic modeling in general, e.g. Cholesky decomposition

**c)** Execute the next cell. You can see three data sets. Which of the three covariance matrices below belongs to which plot?

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize = (20,5))
factors = [0.5,1,0]
names = ["a", "b", "c"]
datasets = []
for i, factor in enumerate(factors):
    x = np.array(np.arange(100).tolist() + np.arange(100).tolist())
    if factor!=0:
        y = x * factor + (np.random.rand(len(x))-0.5)*30
    else:
        y = np.array([50 for _ in range(len(x))]) + (np.random.rand(len(x))-0.5)*30
    plt.subplot(1,len(factors),i+1)
    plt.scatter(x,y)
    plt.title("Dataset "+names[i], fontsize = 30)
    plt.xlabel("x", fontsize = 20)
    plt.ylabel("y", fontsize = 20)
    plt.ylim(0, 100)
    plt.xlim(0, 100)
    datasets.append(np.stack([x, y], axis = 1))
plt.show()

**Covariance matrix I:**

| C1    |  x  |  y  |
|-------|-----|-----|
| **x** | 837 | 877 |
| **y** | 877 | 989 |

**Covariance matrix II:**

 | C2 | x | y |
 | ---|---|---|
 |   **x** |  837 | 1  |
 |   **y**| 1  |  66 |


**Covariance matrix III:**

| C3 | x | y |
| ---|---|---|
|   **x** |  837 | 453  |
|   **y**| 453  |  318 |


Dataset $c$ corresponds to matrix $II$, because of the low $y$ variance and the almost non-existent covariance between $x$ and $y$.  
The $x$ variance is the same for all matrices, but the $y$ variance is very high for $I$ and a lot lower for $III$, which indicates
that dataset $b$ corresponds to matrix $I$ and therefore $a$ to matrix $III$. The higher covariance between $x$ and $y$ in matrix $I$ also confirms that.

**d)** Implement the formula for the autocorrelation matrix

$$ A = \frac{1}{|D|} \sum_{\overrightarrow{x} \in D} (\frac{(\overrightarrow{x} - \overrightarrow{\mu})}{\overrightarrow{s}})^T \frac{(\overrightarrow{x}- \overrightarrow{\mu})}{\overrightarrow{s}} $$

Hint: For your implementation, it might be more efficient to divide by the matrix of standard deviations in the end. The matrix of standard deviations for two dimensions looks like this:

| S | x | y |
| ---|---|---|
|   **x** |  $$s_{x}s_{x}$$| $$s_{x}s_{y}$$ |
|   **y**| $$s_{y}s_{x}$$ |  $$s_{y}s_{y}$$ |


Make sure to make your implementation work for more than two dimensional data as well!

Then execute the cell below which shows the autocorrelation matrices for the datasets of task c.

In [ ]:
def autocorrelation(data):
    """
    Input is an array of the form number_data * dimensions
    An example is shown when this cell is executed
    """
    # YOUR CODE HERE
    
## DELETE COMMENTS TO SHOW THE DATA USED FOR ASSERTION 
# example_x = [2,2,2]
# example_y = [1,2,3]
# example_data = np.stack([[1,2,3], [2,2,2]], axis=1)
# plt.scatter(example_x, example_y)
# plt.show()

test_data1 = np.stack([[1,2,3], [2,2,2]], axis=1)
cor1 = autocorrelation(test_data1)
real_cor1 = [[1.0, 0.0], [0.0, 0.0]]

test_data2 = np.stack([[1,2,3], [4,5,6]], axis=1)
cor2 = autocorrelation(test_data2)
real_cor2 = [[1,1], [1,1]]

#assert np.allclose(cor1, real_cor1), "Wrong output: correlation should be \n {} \n for this data, is \n {}".format(real_cor1, cor1)
assert np.allclose(cor2, real_cor2), "Wrong output: correlation should be \n {} \n for this data, is \n {}".format(real_cor2, cor2)

In [ ]:
# the data of task c is saved in a list called datasets
for i, d in enumerate(datasets):
    print("Autocorrelation matrix for data set", names[i], "\n", autocorrelation(d), "\n")

**e)** What is the difference between the autocorrelation matrix and covariance matrix? When should it be used?

YOUR ANSWER HERE

## Assignment 2: Local PCA (5 Points)

In the lecture we learned that regular PCA is ill suited for special cases of data. In this assignment we will take a look at local PCA which is used for clustered data (ML-06, Slide 25). This is mostly a repetition of algorithms we already used. Feel free to use the built-in functions for k-means clustering and PCA from the libraries (we already included the right imports to set you on track).

In [ ]:
%matplotlib inline

import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt

from numpy.random import multivariate_normal as multNorm

from scipy.cluster.vq import kmeans, vq
from sklearn.decomposition import PCA

np.random.seed(42)

# Generate clustered data - you may plot the data to take a look at it
data = np.vstack((multNorm([2, 2], [[0.1, 0], [0, 1]], 100),
                  multNorm([-2, -4], [[1, 0], [0, 0.3]], 100)))

plt.figure(figsize = (5, 5))
plt.scatter(data[:, 0],data[:, 1])
plt.title("Dataset", fontsize = 15)
plt.xlabel("x", fontsize = 15)
plt.ylabel("y", fontsize = 15)
plt.ylim(-7, 7)
plt.xlim(-7, 7)
plt.show()

# TODO: Apply k-means to the data.
# YOUR CODE HERE
# compute centroids using k-means
centroids, distortion = kmeans(data, 2)
# assign labels to data
labels, dist = vq(data, centroids)

# TODO: Apply PCA for each cluster and store each two largest components.
# YOUR CODE HERE
unique_labels = np.unique(labels)
pcs = {}

for label in unique_labels:
    pca = PCA(n_components=2)
    # perform PCA for cluster
    cluster_data = [data[i] for i in range(len(data)) if labels[i] == label]
    pca.fit(cluster_data)
    pcs[label] = pca

# TODO: Plot the results of k-means and local PCA
# YOUR CODE HERE
def draw_vector(v0, v1, ax=None):
    ax = plt.gca()
    arrowprops = dict(arrowstyle='->', linewidth=3, shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

plt.title("Local PCA", fontsize=15)
plt.xlabel("x", fontsize=15)
plt.ylabel("y", fontsize=15)
plt.plot(centroids[:, 0], centroids[:, 1], '.r', markersize=15)

for label in unique_labels:
    cluster_data = np.array([data[i] for i in range(len(data)) if labels[i] == label])
    plt.scatter(cluster_data[:, 0], cluster_data[:, 1])
    pca = pcs[label]
    for length, vector in zip(pca.explained_variance_, pca.components_):
        v = vector * 3 * np.sqrt(length)
        draw_vector(pca.mean_, pca.mean_ + v)


## Assignment 3: Data Visualization and Chernoff Faces (3 Points)

The following exercise contains no programming (unless you want to go through the implementation). Answer the questions that are posted below the code segment (and run the code before - it's really worth it!). In case you are even more interested - here is a link to the [original paper](http://www.dtic.mil/cgi-bin/GetTRDoc?AD=AD0738473).

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Arc
from numpy.random import rand
import numpy as np

def cface(ax, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18):
    """
    This implementation of chernov faces is taken from Abraham Flaxman. You can
    find the original source files here: https://gist.github.com/aflaxman/4043086
    Only minor adjustments have been made.

     x1 = height  of upper face
     x2 = overlap of lower face
     x3 = half of vertical size of face
     x4 = width of upper face
     x5 = width of lower face
     x6 = length of nose
     x7 = vertical position of mouth
     x8 = curvature of mouth
     x9 = width of mouth
     x10 = vertical position of eyes
     x11 = separation of eyes
     x12 = slant of eyes
     x13 = eccentricity of eyes
     x14 = size of eyes
     x15 = position of pupils
     x16 = vertical position of eyebrows
     x17 = slant of eyebrows
     x18 = size of eyebrows
    """

    # transform some values so that input between 0,1 yields variety of output
    x3 = 1.9 * (x3 - .5)
    x4 = (x4 + .25)
    x5 = (x5 + .2)
    x6 = .3 * (x6 + .01)
    x8 = 5 * (x8 + .001)
    x11 /= 5
    x12 = 2 * (x12 - .5)
    x13 += .05
    x14 += .1
    x15 = .5 * (x15 - .5)
    x16 = .25 * x16
    x17 = .5 * (x17 - .5)
    x18 = .5 * (x18 + .1)

    # top of face, in box with l=-x4, r=x4, t=x1, b=x3
    e = Ellipse((0, (x1 + x3) / 2), 2 * x4, (x1 - x3), ec='black', linewidth=2)
    ax.add_artist(e)

    # bottom of face, in box with l=-x5, r=x5, b=-x1, t=x2+x3
    e = Ellipse((0, (-x1 + x2 + x3) / 2), 2 * x5, (x1 + x2 + x3), fc='white', ec='black', linewidth=2)
    ax.add_artist(e)

    # cover overlaps
    e = Ellipse((0, (x1 + x3) / 2), 2 * x4, (x1 - x3), fc='white', ec='none')
    ax.add_artist(e)
    e = Ellipse((0, (-x1 + x2 + x3) / 2), 2 * x5, (x1 + x2 + x3), fc='white', ec='none')
    ax.add_artist(e)

    # draw nose
    plt.plot([0, 0], [-x6 / 2, x6 / 2], 'k')

    # draw mouth
    p = Arc((0, -x7 + .5 / x8), 1 / x8, 1 / x8, theta1=270 - 180 / np.pi * np.arctan(x8 * x9),
            theta2=270 + 180 / np.pi * np.arctan(x8 * x9))
    ax.add_artist(p)

    # draw eyes
    p = Ellipse((-x11 - x14 / 2, x10), x14, x13 * x14, angle=-180 / np.pi * x12, fc='white', ec='black')
    ax.add_artist(p)

    p = Ellipse((x11 + x14 / 2, x10), x14, x13 * x14, angle=180 / np.pi * x12, fc='white', ec='black')
    ax.add_artist(p)

    # draw pupils
    p = Ellipse((-x11 - x14 / 2 - x15 * x14 / 2, x10), .05, .05, facecolor='black')
    ax.add_artist(p)
    p = Ellipse((x11 + x14 / 2 - x15 * x14 / 2, x10), .05, .05, facecolor='black')
    ax.add_artist(p)

    # draw eyebrows
    ax.plot([-x11 - x14 / 2 - x14 * x18 / 2, -x11 - x14 / 2 + x14 * x18 / 2],
            [x10 + x13 * x14 * (x16 + x17), x10 + x13 * x14 * (x16 - x17)], 'k')
    ax.plot([x11 + x14 / 2 + x14 * x18 / 2, x11 + x14 / 2 - x14 * x18 / 2],
            [x10 + x13 * x14 * (x16 + x17), x10 + x13 * x14 * (x16 - x17)], 'k')


fig, axes = plt.subplots(nrows=5, ncols=5, num='Chernoff Faces', figsize=(9, 9))
for ax in axes.flat:
    cface(ax, .9, *rand(17))
    ax.axis([-1.2, 1.2, -1.2, 1.2])
    ax.set(xticks=[], yticks=[])

fig.subplots_adjust(hspace=0, wspace=0)


### a) Data Visualization Techniques

Why do we need data visualization techniques and what are techniques to visualize high dimensional data?

Data visualization techniques make complex data more accessible, understandable and usable for humans.  
One technique to visualize high dimensional data would be PCA, it is often able to make hidden structure in the data visible.  
Howevery, PCA is not always able to visualize the data in an appropriate way. We have seen a couple of techniques in the lecture that can help when trying to visualize high dimensional data such as glyphs, scatterplot matrices and other projection techniques besides PCA.  
If possible, it's often a good idea to project the data in a lower dimensional space, but this can lead to an information loss which it not always feasible.

### b) Chernoff faces

Why did Chernoff use faces for his representation? Why not something else, like dogs or houses?

The human visual system is particularly sensitive for faces, i.e. we are very good at interpreting and recognizing human faces.  
Chernoff used faces to make use of this property when representing high dimensional data.

### c) Alternatives

Explain at least one other data visualization technique from the lecture.

- **Scatterplot matrix**: Project on two of the dimensions and display all combinations as matrix of 2D plots
- **Glyphs**: Map each dimension onto the parameters of a geometrical figure
- **Projection techniques**: Do not represent the complete information, but project onto selected directions